In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 18:05:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/24 18:05:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/24 18:05:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Question 1

Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

- 3.3.2 (+)
- 2.1.4
- 1.2.3
- 5.4

In [2]:
spark.version

'3.3.2'

# Question 2

HVFHW June 2021

Read it with Spark using the same schema as we did in the lessons.

We will use this dataset for all the remaining questions.

Repartition it to 12 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 2MB
- 24MB (+)
- 100MB
- 250MB 

In [3]:
!ls

01_spark_init.ipynb  fhvhv_tripdata_2021-06.csv.gz
02_homework.ipynb    taxi+_zone_lookup.csv


In [15]:
import pandas as pd

In [18]:
pd.read_csv('fhvhv_tripdata_2021-06.csv.gz', nrows=5)

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [19]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

In [31]:
schema = StructType([
    StructField('dispatching_base_num', StringType(), True), 
    StructField('pickup_datetime', TimestampType(), True), 
    StructField('dropoff_datetime', TimestampType(), True), 
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True), 
    StructField('SR_Flag', StringType(), True), 
    StructField('Affiliated_base_number', StringType(), True)
])

In [32]:
df = spark.read.schema(schema).csv('fhvhv_tripdata_2021-06.csv.gz', header=True)

In [33]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [28]:
df.repartition(12).write.parquet('fhvhv_tripdata_2021_06.parquet')

23/02/24 18:23:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag, Affiliated_base_number
 Schema: dispatching_base_num, pickup_datetime, dropoff_datetime, pickup_location_id, dropoff_location_id, sr_flag, affiliated_base_number
Expected: pickup_location_id but found: PULocationID
CSV file: file:///home/oleg/spark/fhvhv_tripdata_2021-06.csv.gz


In [29]:
!ls -lh fhvhv_tripdata_2021_06.parquet

total 284M
-rw-r--r-- 1 oleg oleg   0 Feb 24 18:25 _SUCCESS
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:24 part-00000-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:24 part-00001-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:24 part-00002-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:24 part-00003-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:25 part-00004-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:25 part-00005-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:24 part-00006-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:25 part-00007-ba4f04e8-7733-4463-894e-14fe852248cb-c000.snappy.parquet
-rw-r--r-- 1 oleg oleg 24M Feb 24 18:25 part-00008-ba4f04e8-7733-4463-894e-1

# Question 3

Count records.

How many taxi trips were there on June 15?

Consider only trips that started on June 15.

- 308,164
- 12,856
- 452,470 (+)
- 50,982 

In [35]:
df.show(1, vertical=True)

-RECORD 0-------------------------------------
 dispatching_base_num   | B02764              
 pickup_datetime        | 2021-06-01 00:02:41 
 dropoff_datetime       | 2021-06-01 00:07:46 
 PULocationID           | 174                 
 DOLocationID           | 18                  
 SR_Flag                | N                   
 Affiliated_base_number | B02764              
only showing top 1 row



In [36]:
import pyspark.sql.functions as F

In [38]:
(
    df
    .filter(F.col('pickup_datetime').between('2021-06-15', '2021-06-15 23:59:59'))
    .count()
)

452470

# Question 4

Longest trip for each day.

Now calculate the duration for each trip. How long was the longest trip in Hours?

- 66.87 Hours (+)
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

In [45]:
(
    df
    .select('dispatching_base_num', 'pickup_datetime', 'dropoff_datetime')
    .withColumn(
        'trip_duration', 
        (F.unix_timestamp('dropoff_datetime') - F.unix_timestamp('pickup_datetime')) / 60 / 60
    )
    .agg(F.max('trip_duration'))
).show()

+------------------+
|max(trip_duration)|
+------------------+
| 66.87888888888888|
+------------------+



# Question 5

User Interface.

Spark’s User Interface which shows application's dashboard runs on which local port?

- 80
- 443
- 4040 (+)
- 8080

In [47]:
spark.sparkContext

<SparkContext master=local[*] appName=test>

# Question 6

Most frequent pickup location zone.

Load the zone lookup data into a temp view in Spark.

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

- East Chelsea
- Astoria
- Union Sq
- Crown Heights North (+)

In [51]:
schema_zones = StructType([
    StructField('LocationID', IntegerType(), True),
    StructField('Borough', StringType(), True),
    StructField('Zone', StringType(), True),
    StructField('service_zone', StringType(), True),
])

In [52]:
df_zones = spark.read.schema(schema_zones).csv('taxi+_zone_lookup.csv', header=True)

In [53]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [54]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [55]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [59]:
(
    df
    .join(df_zones, F.col('PULocationID') == F.col('LocationID'), 'left')
    .groupBy('Zone')
    .agg(F.count('pickup_datetime').alias('trips'))
    .orderBy(-F.col('trips'))
).show(5)

+-------------------+------+
|               Zone| trips|
+-------------------+------+
|Crown Heights North|231279|
|       East Village|221244|
|        JFK Airport|188867|
|     Bushwick South|187929|
|      East New York|186780|
+-------------------+------+
only showing top 5 rows

